#  triple extraction task on GM-CIHT,DDI,Chemprot

In [2]:
# huggingface-cli login

# from huggingface_hub import login
# login()
# I was having the same issue in Jupyter and I was able to log in with:
from huggingface_hub import interpreter_login
 
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ········
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termi

## 提取DDI中的relation

In [3]:
import json

relations=[]
with open("dataset/1_DDI/train_triples.json") as fr:
    for line in fr.readlines():
        line=json.loads(line.strip())
        for li in line:
            relations.append(li["triple_list"][0][1]) #

print(set(relations))

{'mechanism', 'int', 'None', 'effect', 'advise'}


In [15]:
sentences=[]
with open("dataset/1_DDI/train_triples.json") as fr:
    for line in fr.readlines():
        line=json.loads(line.strip())
        for li in line:
            if li["triple_list"][0][1]!="None":
                sentences.append("context: "+li["text"] + "response: "+ "|".join(li["triple_list"][0]))

print(len(sentences))
print(len(set(sentences)))

1027
1010


## generate embadding for each sentences in training set

In [5]:
import torch
import json
from transformers import AutoTokenizer, AutoModel
import numpy as np


sentences=[]
with open("dataset/1_DDI/train_triples.json") as fr:
    for line in fr.readlines():
        line=json.loads(line.strip())
        for li in line:
            if li["triple_list"][0][1]!="None":
                sentences.append("context: "+li["text"] + "response: "+ "|".join(li["triple_list"][0]))

            
print('1'*100)
# print(sentences)

tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

print('2'*100)


# Apply tokenizer
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
print('3'*100)


# Compute token embeddings
# outputs = model(**inputs)
# print(outputs)
# print('4'*100)


# Mean pooling

# Apply tokenizer
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings



all_sentence_vector=[]
l=-1
for sentence in sentences:
    l=l+1
    print(l)
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    outputs = model(**inputs)
    # Mean pooling
    embeddings = mean_pooling(outputs[0], inputs['attention_mask'])
    np_embedding=embeddings.detach().numpy()[0]
    all_sentence_vector.append(np_embedding)


np.save("train_embedding_DDI.npy", all_sentence_vector)

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222
3333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201

## generate training instruction data

In [7]:
import torch
import json
from transformers import AutoTokenizer, AutoModel
import numpy as np


sentences=[]
with open("dataset/1_DDI/train_triples.json") as fr:
    for line in fr.readlines():
        line=json.loads(line.strip())
        for li in line:
            if li["triple_list"][0][1]!="None":
                sentences.append("context: "+li["text"] + "response: "+ "|".join(li["triple_list"][0]))

                
Stored_Embeddings=np.load("train_embedding_DDI.npy")


tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

def get_cos_similar(v1: list, v2: list):
    num = float(np.dot(v1, v2))  # 向量点乘
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)  # 求模长的乘积
    return 0.5 + 0.5 * (num / denom) if denom != 0 else 0

# Mean pooling

# Apply tokenizer
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

# instruction = f"You are an excellent linguist. The task is to predict the  relationship between the given head entity and tail entity in a sentence, this relation  must be in ('PREDISPOSES', 'DIAGNOSES', 'INTERACTS_WITH', 'ADMINISTERED_TO', 'ASSOCIATED_WITH', 'STIMULATES', 'AFFECTS', 'PREVENTS', 'USES', 'CAUSES', 'TREATS', 'PROCESS_OF')"
instruction = f"please extract the triplet from this sentence, the triplet is [head entity, relation, tail entity], \
the element relation denotes the relationship between head entity and tail entity, I will provide you \
the definition of the triplet you need to extract, the sentence from where your extract the triplets \
(head entity, relation, tail_entity) and the output format with examples. the relation must in my \
predefined relation set: ('effect', 'advise', 'mechanism', 'int').  \
response Format: head entity|relation|tail entity."

fw=open("DDI_train_instruction_container.json", "w")

h=0
with open("dataset/1_DDI/train_triples.json") as fr:
    for line in fr.readlines():
        line = json.loads(line.strip())
        for li in line:
            if li["triple_list"][0][1]=="None":
                continue
            h=h+1
            print(h)
            sentence="context: " + li["text"]
            inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
                # Compute token embeddings
            outputs = model(**inputs)
            # Mean pooling
            embeddings = mean_pooling(outputs[0], inputs['attention_mask'])
            np_embedding=embeddings.detach().numpy()[0]
            all_cos=[]
            for i in range(len(Stored_Embeddings)):
                cos_=get_cos_similar(np_embedding,Stored_Embeddings[i])
                all_cos.append(cos_)
                
            arr = np.array(all_cos)
#             max_=arr.argsort()[-1:][::-1][0]
            max_indices = np.where(arr == np.max(arr))[0].tolist()
            Examples = [sentences[idx] for idx in max_indices]

#             Excample=sentences[max_index]
            # print(excamples_)
            # print(excamples_)


            Dic_ = {}
            Dic_["instruction"] = instruction + " Excample: " + " Excample:".join(Examples)
            Dic_["context"] = li["text"]
            Dic_["response"] = "|".join(li["triple_list"][0])

            Dic_["category"] = "triplet extraction"

            fw.write(json.dumps(Dic_))
            fw.write("\n")
#             if len(Examples)>1:
#                 print()
#                 print()
#                 print()
#                 print()
#                 print()
#                 print()
#                 print()
#             print(Dic_)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## generating test instruction data

In [8]:
import torch
import json
from transformers import AutoTokenizer, AutoModel
import numpy as np


sentences=[]
with open("dataset/1_DDI/train_triples.json") as fr:
    for line in fr.readlines():
        line=json.loads(line.strip())
        for li in line:
            if li["triple_list"][0][1]!="None":
                sentences.append("context: "+li["text"] + "response: "+ "|".join(li["triple_list"][0]))

                
Stored_Embeddings=np.load("train_embedding_DDI.npy")


tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

def get_cos_similar(v1: list, v2: list):
    num = float(np.dot(v1, v2))  # 向量点乘
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)  # 求模长的乘积
    return 0.5 + 0.5 * (num / denom) if denom != 0 else 0

# Mean pooling

# Apply tokenizer
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

# instruction = f"You are an excellent linguist. The task is to predict the  relationship between the given head entity and tail entity in a sentence, this relation  must be in ('PREDISPOSES', 'DIAGNOSES', 'INTERACTS_WITH', 'ADMINISTERED_TO', 'ASSOCIATED_WITH', 'STIMULATES', 'AFFECTS', 'PREVENTS', 'USES', 'CAUSES', 'TREATS', 'PROCESS_OF')"
instruction = f"please extract the triplet from this sentence, the triplet is [head entity, relation, tail entity], \
the element relation denotes the relationship between head entity and tail entity, I will provide you \
the definition of the triplet you need to extract, the sentence from where your extract the triplets \
(head entity, relation, tail_entity) and the output format with examples. the relation must in my \
predefined relation set: ('effect', 'advise', 'mechanism', 'int').  \
response Format: head entity|relation|tail entity."

fw=open("DDI_test_instruction_container.json", "w")

h=0
with open("dataset/1_DDI/test_triples.json") as fr:
    for line in fr.readlines():
        line = json.loads(line.strip())
        for li in line:
            if li["triple_list"][0][1]=="None":
                continue
            h=h+1
            print(h)
            sentence="context: " + li["text"]
            inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
                # Compute token embeddings
            outputs = model(**inputs)
            # Mean pooling
            embeddings = mean_pooling(outputs[0], inputs['attention_mask'])
            np_embedding=embeddings.detach().numpy()[0]
            all_cos=[]
            for i in range(len(Stored_Embeddings)):
                cos_=get_cos_similar(np_embedding,Stored_Embeddings[i])
                all_cos.append(cos_)
                
            arr = np.array(all_cos)
#             max_=arr.argsort()[-1:][::-1][0]
            max_indices = np.where(arr == np.max(arr))[0].tolist()
            Examples = [sentences[idx] for idx in max_indices]

#             Excample=sentences[max_index]
            # print(excamples_)
            # print(excamples_)


            Dic_ = {}
            Dic_["instruction"] = instruction + " Excample: " + " Excample:".join(Examples)
            Dic_["context"] = li["text"]
            Dic_["response"] = "|".join(li["triple_list"][0])

            Dic_["category"] = "triplet extraction"

            fw.write(json.dumps(Dic_))
            fw.write("\n")
#             if len(Examples)>1:
#                 print()
#                 print()
#                 print()
#                 print()
#                 print()
#                 print()
#                 print()
#             print(Dic_)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Trian Llama-13b

In [10]:
import json
with open("DDI_train_instruction_container.json") as fr:
    line = fr.readlines()
    print(line)

['{"instruction": "please extract the triplet from this sentence, the triplet is [head entity, relation, tail entity], the element relation denotes the relationship between head entity and tail entity, I will provide you the definition of the triplet you need to extract, the sentence from where your extract the triplets (head entity, relation, tail_entity) and the output format with examples. the relation must in my predefined relation set: (\'effect\', \'advise\', \'mechanism\', \'int\').  response Format: head entity|relation|tail entity. Excample: context: A case report of one patient taking amiodarone 200 mg and indinavir 800 mg three times a day resulted in increases in amiodarone concentrations from 0.9 mg/L to 1.3 mg/L.response: amiodarone|mechanism|indinavir", "context": "A case report of one patient taking amiodarone 200 mg and indinavir 800 mg three times a day resulted in increases in amiodarone concentrations from 0.9 mg/L to 1.3 mg/L.", "response": "amiodarone|mechanism|in

In [ ]:
import json
with open("DDI_test_instruction_container.json") as fr:
    line = fr.readlines()
    print(line)

# from datasets import load_dataset
import torch
import transformers
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from transformers import LlamaTokenizer
from trl import SFTTrainer
from datasets import load_dataset

print("1"*50)


def formatting_func(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n"
      f"{example['response']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['response']}")

  return {"text" : input_prompt}

def prepare_data(path):
    data = load_dataset("json", data_files=path)
    formatted_data = data.map(formatting_func)
    # print( formatted_data["train"])
    return formatted_data["train"]


# formatted_dataset = dbricks_15k_dataset_prepared.map(formatting_func)


train_path="DDI_train_instruction_container.json"
test_path="DDI_test_instruction_container.json"
# test_path="test_chuck_instruction_5_llama13b_right.json"

train = prepare_data(train_path)
test = prepare_data(test_path)

# print(train["train"][2]["text"])
print("2"*50)

# print(train[5])
# print()
# print(test[5])



# # # model_id = '/scratch/ahcie-gpu2/openllama-models/MedLLaMA_13B'
# # model_id="/scratch/ahcie-gpu2/llama-models-meta-hf/Llama-2-13b-hf"
model_id = "meta-llama/Llama-2-13b-hf"
# model_id = "medalpaca/medalpaca-13b"

qlora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,torch_dtype=torch.float16, device_map='auto',
)

print("3"*50)


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# tokenizer.add_special_tokens({'end_token': '[END]'})

print("4"*50)

supervised_finetuning_trainer = SFTTrainer(
    base_model,
    train_dataset=train,
    eval_dataset=test,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        max_steps=5000,
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        output_dir="Our_model",
        optim="paged_adamw_8bit",
        fp16=True,
        evaluation_strategy = "steps",
        eval_steps = 50,
        save_steps = 50,
        load_best_model_at_end=True,
        save_strategy='steps',
    ),
    tokenizer=tokenizer,
    peft_config=qlora_config,
    dataset_text_field="text",
    max_seq_length=128 #4098
)

supervised_finetuning_trainer.train()